In [ ]:
# Calculate distance
import geopandas as gpd

features_file = "Class_1.shp"
roads_file = "grid.geojson"
output_file = "Class_1_distance.shp"

features_gdf = gpd.read_file(features_file)
roads_gdf = gpd.read_file(roads_file)

if features_gdf.crs != roads_gdf.crs:
    roads_gdf = roads_gdf.to_crs(features_gdf.crs)

projected_crs = "EPSG:32647"
features_gdf = features_gdf.to_crs(projected_crs)
roads_gdf = roads_gdf.to_crs(projected_crs)

def calculate_nearest_distance(row, roads):
    """
    Calculate the nearest distance from the current feature to the roads.
    """
    return roads.distance(row.geometry).min()

features_gdf["nearest_dist"] = features_gdf.apply(calculate_nearest_distance, roads=roads_gdf, axis=1)

features_gdf = features_gdf.rename(columns={"nearest_dist": "near_dist"})

features_gdf.to_file(output_file)
print(f"Distance calculation completed. Output saved to: {output_file}")

In [ ]:
# shp to csv for further calculation
import geopandas as gpd

input_shapefile = "Class_1_distance.shp"  
output_csv = "Class_1_distance.csv"

gdf = gpd.read_file(input_shapefile)
gdf.to_csv(output_csv, index=False)

print(f"Shapefile saved to CSV.: {output_csv}")

In [ ]:
# Least-cost optimisation
import pandas as pd
import numpy as np

file_path = "Economic_Calculation.xlsx"
sheet_name = "Sheet1"
data = pd.read_excel(file_path, sheet_name=sheet_name)

data["pwr_mw"] = data["pwr_kw"] / 1000
target_power_mw = 72130  # Target total power generation (MW)

n = len(data)
max_power = int(target_power_mw)
dp = np.full((n + 1, max_power + 1), np.inf) 
dp[0, 0] = 0 

for i in range(1, n + 1):
    for j in range(max_power + 1):
        dp[i, j] = dp[i - 1, j]
        power = int(data["pwr_mw"].iloc[i - 1])
        cost = data["Install_price_Million USD"].iloc[i - 1]
        if j >= power:
            dp[i, j] = min(dp[i, j], dp[i - 1, j - power] + cost)

for power_needed in range(max_power, -1, -1):
    if dp[n, power_needed] != np.inf:
        best_price = dp[n, power_needed]
        target_reached = power_needed
        break
else:
    print("Unable to meet the power generation target.")
    exit()

selected_rows = []
remaining_power = target_reached
for i in range(n, 0, -1):
    if remaining_power >= 0 and dp[i, remaining_power] != dp[i - 1, remaining_power]:
        selected_rows.append(i - 1)
        remaining_power -= int(data["pwr_mw"].iloc[i - 1])

result_df = data.iloc[selected_rows].sort_index()
result_df.to_excel("New solar power plants location.xlsx", index=False)

print(f"Selected projects, total power: {result_df['pwr_mw'].sum()} MW, total installation cost: {best_price:.2f} million USD")
print("Results saved to: New solar power plants location.xlsx")

In [ ]:
# Filter out the final location
import pandas as pd
import geopandas as gpd

excel_file = "New_solar_power_plants_location.xlsx"
df = pd.read_excel(excel_file)

shapefile_input = "Class_1_power.shp"
gdf = gpd.read_file(shapefile_input)

common_fids = df["FID"].unique()
filtered_gdf = gdf[gdf["FID"].isin(common_fids)]

if filtered_gdf.empty:
    print("No matching FIDs found. No new Shapefile generated.")
else:
    output_shapefile = "New_solar_power_plants.shp"
    filtered_gdf.to_file(output_shapefile)
    print(f"Extraction completed. New Shapefile saved at: {output_shapefile}")